<style>
    h1 {
    font-family: 'Segoe UI Semibold';
    font-size: 35px;
    text-align: center;
  }
</style>

# Taller 9: Bases de Datos en la Nube (RDS)


<style>
  .image {
            overflow: hidden;
            width: 100%;
            border-radius: 20px;
            object-fit: cover;
            object-position: top center;
        }
  .container {
            width: 100%;
            height: 350px;
        }

</style>
<div class="container">
    <img src= "./img/sql-cloud.png" class="image"/>
</div>

<style>
  p {
    font-family: 'Segoe UI Light';
    font-size: 20px;
    text-align: center;
  }
</style>

<div style="text-align: center;">
  Analítica Computacional para la Toma de Decisiones IIND-4130<br>
  Universidad de los Andes &copy; 2024 <br>
  21 de octubre de 2024 <br><br>
  <strong>GRUPO:</strong> <br>
  Camilo Duque - 202024289<br>
  Sofia Buitrago – 202020742<br>
</div>


### **Importar Librerias**

In [4]:
import psycopg2

### **Connect a Base de Datos**

In [9]:
engine = psycopg2.connect(
dbname="bd_postgres",
user="cduquec",
password="1234Camilo",
host="database-camilo.c1y0em2ay3hc.us-east-1.rds.amazonaws.com",
port='5432'
)